In [1]:
from fastai import *
import fastai
from fastai.text.all import *
# from fastai.text import *
# from fastai.test_utils import *
from pathlib import Path
import pandas as pd
import numpy as np
import re

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
path = Path('./data')
path_lang_model = Path('./data/lang_model_data')

# sample2 = pd.read_csv(path/'sampl.csv')
# sample3 = pd.read_csv(path/'sample_3.csv')

In [4]:
sample1 = pd.read_csv(path/'sample_1.csv')

In [5]:
sample1['stance'][0] = 'A'
sample1['stance'].fillna('N', inplace=True)
sample1=sample1.iloc[:, 1:]

/tmp/ipykernel_11143/327552301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample1['stance'][0] = 'A'


In [6]:
sample1['stance'].isnull().sum()

0

In [7]:
sample2 = pd.read_csv(path/'sample_2_-_sample_2.csv', encoding='utf-8')
sample2 = sample2.iloc[:, 1:]
sample2.columns = ['tweetId','renderedContent','stance']

In [8]:
sample3 = pd.read_csv(path/'sample_3final.csv', encoding='utf-8')
sample3 = sample3.iloc[:, 1:]

In [9]:
sample3.head()

,tweetId,renderedContent,stance
0,1.360000e+18,"I wonder those who brought day 3/4/5 tickets, are they going to be reimbursed or is Modi going to take that money and invest it in more barricades against the farmers 🤔 #FarmersProtest #INDvsENG #NarendraModiStadium #ModiTerrorist",F
1,1.330000e+18,I really want to get into independent journalism pained at seeing the distorted coverage by the paid so-called mainstream news outlets😢 Will their sleeping conscience ever wake up? #SaveDemocracy #GodiMedia #FarmersProtest,F
2,1.340000e+18,It's for everyone's good #WeUnitedForFarmers #FarmersProtest Repeal black farm laws …,F
3,1.410000e+18,"No farmers , no food ! #मोदी_ही_डकैत_है #FarmersProtest …",F
4,1.350000e+18,The farmer unions are likely to announce the cancellation of the march to #Parliament on February 1 | #FarmersProtest …,N


In [10]:
sample2.head()

,tweetId,renderedContent,stance
0,1.41E+18,India needs a govt of law not of a man repeal the bills #FarmersProtest_AtParliament #FarmersProtest,F
1,1.38E+18,That's how corporates and govt. together looting the farmers #FarmersProtest #Modi_Sold_Farmers #Modi_MSP_DO …,F
2,1.42E+18,Every tweet Every rt Every post matters Please keep supporting farmers #FarmersProtest,F
3,1.36E+18,"Millions and Millions of lives Don’t matter , Farmers lives are less important to BJP They just want to Sell us OUT #RepealOnlyWayAhead #FarmersProtest #RepealOnlyWayAhead",F
4,1.36E+18,Repeal black farm laws #FencingLikeChinaPak #FarmersProtest …,F


In [11]:
sample1.head()

,tweetId,renderedContent,stance
0,1.350000e+18,"LoL this tactic should have been tried much before, the 'kisan' committies are not united as they seem. Fake #FarmersProtest …",A
1,1.340000e+18,Farmers Protest in Delhi | Giant Panther #FarmersProtest #Agriculture #VivasayigalPorattam #AgricultureBill2020 #AgricultureOrdinance #FarmerProtestDelhi #GiantPanther #AnjaneyaProductions 1otWgWA,A
2,1.360000e+18,We stand with farmers #RememberSirChhotuRamInModiEra #FarmersProtest,N
3,1.330000e+18,today is indian constitution day and farmers doing protest. i support farmers. #FarmersProtest,F
4,1.350000e+18,Kuch yaad aya? Ravi Nair () Tweeted: Yeh inquilab hai sir. This is a revolution...👇 #FarmersProtest (…),F


In [12]:
sample1['stance'].value_counts()

F    1315
N     512
A     164
S       3
T       2
y       1
P       1
t       1
s       1
Name: stance, dtype: int64

In [13]:
def rename_stance(stance):
    if stance == 'F':
        return 'F'
    elif stance == 'A':
        return 'A'
    else:
        return 'N'

In [14]:
sample1['stance'] = sample1['stance'].apply(rename_stance)
sample1['stance'].value_counts()

F    1315
N     521
A     164
Name: stance, dtype: int64

In [15]:

sample2['stance'] = sample2['stance'].apply(rename_stance)
sample2['stance'].value_counts()

F    1595
N     292
A     113
Name: stance, dtype: int64

In [16]:
sample3['stance'] = sample3['stance'].apply(rename_stance)
sample3['stance'].value_counts()

F    1395
N     547
A      58
Name: stance, dtype: int64

In [17]:
df = pd.concat([sample1, sample2, sample3])

df = df.sample(frac = 1).reset_index(drop=True)

In [18]:
df.head()

,tweetId,renderedContent,stance
0,1370000000000000000.0,#FarmersProtest Is India A Democracy dont think so. …,F
1,1370000000000000000.0,God Bless each and every person supporting us #PunishAntiFarmerBJP #FarmersProtest 8T2Vjh0CRuuRIL…,F
2,1.38E+18,"A story by for . Canada India Foundation, the organization which outrightly denies the Sikh genocide & openly campaigns for Modi’s Hindutva govt has started a petition to stop Canadian schools from talking about #FarmersProtest. …",N
3,1430000000000000000.0,Latest twitter trends in #India: 1. #भगवदगीता_के_गूढ़_रहस्य 2. #LIVCHE 3. #किसान_विरोधी_खट्टर 4. #INDvsEND 5. #FarmersProtest 6. Nehru 7. Anthony Taylor 8. Ronaldo 9. Arsenal 10. SSRians Roar At CBI HQ,N
4,1.39E+18,#COVID19India #NoMoreModi Rab kher kare Sleeping are awake #OcygenShortage #VaccineShortage #KisanCovidHelp #LangarOxygen #farmlaws_nextpandemic #TakeBack_FarmLaws #FarmersProtest,N


In [19]:
df.drop_duplicates(subset='renderedContent', inplace=True, keep='first')

In [20]:
len(df)

5965

In [21]:
df = df[['renderedContent', 'stance']]
# df.columns = ['text', 'category']

##### df['stance'].value_counts()


In [22]:
valid = np.random.choice([False,True], len(df['renderedContent']), p= [0.9, 0.1])
df['is_valid'] = valid
df.head()

,renderedContent,stance,is_valid
0,#FarmersProtest Is India A Democracy dont think so. …,F,False
1,God Bless each and every person supporting us #PunishAntiFarmerBJP #FarmersProtest 8T2Vjh0CRuuRIL…,F,False
2,"A story by for . Canada India Foundation, the organization which outrightly denies the Sikh genocide & openly campaigns for Modi’s Hindutva govt has started a petition to stop Canadian schools from talking about #FarmersProtest. …",N,True
3,Latest twitter trends in #India: 1. #भगवदगीता_के_गूढ़_रहस्य 2. #LIVCHE 3. #किसान_विरोधी_खट्टर 4. #INDvsEND 5. #FarmersProtest 6. Nehru 7. Anthony Taylor 8. Ronaldo 9. Arsenal 10. SSRians Roar At CBI HQ,N,False
4,#COVID19India #NoMoreModi Rab kher kare Sleeping are awake #OcygenShortage #VaccineShortage #KisanCovidHelp #LangarOxygen #farmlaws_nextpandemic #TakeBack_FarmLaws #FarmersProtest,N,False


In [162]:
df_pos = df[df['stance'] == 'F']
df_neg = df[df['stance'] == 'A']
df_neutral = df[df['stance'] == 'N']

In [163]:
df_pos = df_pos[:1000]

df_neutral = df_neutral[:1000]

df_test = pd.concat([df_pos, df_neutral, df_neg])
df_test = df_test.sample(frac = 1).reset_index(drop=True)

In [25]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
train_path = Path(path_lang_model, 'train.csv')
test_path = Path(path_lang_model, 'test.csv')

# save the train and test file
# again using the '\t' separator to create tab-separated-values files
train.to_csv(train_path)
test.to_csv(test_path)

In [26]:
len(train)

4772

In [27]:
len(test)

1193

In [28]:
lang_df = pd.read_csv(path/'final_cleaned_data.csv')
bs = 20
lang_df=lang_df.iloc[:, 1:]
lang_df.head()

,text
0,Supporting farmers means supporting our country #StopHateAgainstFarmers #FarmersProtest
1,Support farmers if you are related to food #StopHateAgainstFarmers #FarmersProtest
2,#StopHateAgainstFarmers support #FarmersProtest by pressurising got. to accept farmer's genuine demand.
3,"You hate farmers I hate you, if you love them I love you! ;) #StopHateAgainstFarmers #FarmersProtest"
4,They can't be farmers. Looks like Gundas are having good time. They seem to be violence thirsty goons. #FarmersProtest …


In [29]:
valid = np.random.choice([False,True], len(lang_df['text']), p= [0.9, 0.1])
lang_df['is_valid'] = valid
lang_df.head()

,text,is_valid
0,Supporting farmers means supporting our country #StopHateAgainstFarmers #FarmersProtest,False
1,Support farmers if you are related to food #StopHateAgainstFarmers #FarmersProtest,False
2,#StopHateAgainstFarmers support #FarmersProtest by pressurising got. to accept farmer's genuine demand.,False
3,"You hate farmers I hate you, if you love them I love you! ;) #StopHateAgainstFarmers #FarmersProtest",True
4,They can't be farmers. Looks like Gundas are having good time. They seem to be violence thirsty goons. #FarmersProtest …,False


In [30]:
data_lm = DataBlock(blocks=TextBlock.from_df('text', is_lm=True),
                    get_x=ColReader('text'),
                    splitter=ColSplitter())
dls = data_lm.dataloaders(lang_df, bs=bs, seq_len=72)
dls.show_batch(max_n=6)

,text,text_
0,"xxbos xxmaj stop embarrassing xxmaj india , farmers want bills repealed , to protect livelihoods . xxmaj modi mafia is using undemocratic , dangerous tactis . xxmaj world is watching . # farmersprotest # justicefornavreet xxbos xxmaj no farmer xxmaj no xxunk # farmersprotest … xxbos never saw any punjabi female from xxmaj canada tweeting on # farmersprotest . xxmaj but you are xxup first . xxmaj keep it xxmaj up .","xxmaj stop embarrassing xxmaj india , farmers want bills repealed , to protect livelihoods . xxmaj modi mafia is using undemocratic , dangerous tactis . xxmaj world is watching . # farmersprotest # justicefornavreet xxbos xxmaj no farmer xxmaj no xxunk # farmersprotest … xxbos never saw any punjabi female from xxmaj canada tweeting on # farmersprotest . xxmaj but you are xxup first . xxmaj keep it xxmaj up . xxbos"
1,"जीतेगा_तो_किसान_ही when # चौकीदार_ही_चोर_है # farmersprotest is also against the misuse of xxmaj constitutional power taken by our mandate on buff & xxmaj hoax . # indiabeingsilenced by xxmaj police xxbos xxmaj they are just asking for their right to survive . # farmersprotestdays # farmersprotest … xxbos 🎶 "" work "" 🎶 # farmersprotest # rihannasupportsindianfarmers # rihannasoldout xxbos xxup msp will serve several purposes 👇 🏼 # farmersshineinolympics # farmersprotest","when # चौकीदार_ही_चोर_है # farmersprotest is also against the misuse of xxmaj constitutional power taken by our mandate on buff & xxmaj hoax . # indiabeingsilenced by xxmaj police xxbos xxmaj they are just asking for their right to survive . # farmersprotestdays # farmersprotest … xxbos 🎶 "" work "" 🎶 # farmersprotest # rihannasupportsindianfarmers # rihannasoldout xxbos xxup msp will serve several purposes 👇 🏼 # farmersshineinolympics # farmersprotest /"
2,आज का hashtag retweet everyone # shaheedbhagatsingh # farmersprotest xxbos # farmersprotest # bhagodamodi # farmersbill # merryxmas # मनुस्मृति_दहन_दिवस # जोकर_नहीं_देशभक्त_बनो # किसानों_का_दर्द_समझो # किसान_एकता_जिंदाबाद xxmaj follow 👉 💯 % follow back xxbos # farmersprotest # xxmaj twitter xxmaj indian govt want xxmaj twitter to remove tweets / accounts which substantially supports xxmaj protest . i really needed to use aggressive words but aware of twitter violence … shame on your,का hashtag retweet everyone # shaheedbhagatsingh # farmersprotest xxbos # farmersprotest # bhagodamodi # farmersbill # merryxmas # मनुस्मृति_दहन_दिवस # जोकर_नहीं_देशभक्त_बनो # किसानों_का_दर्द_समझो # किसान_एकता_जिंदाबाद xxmaj follow 👉 💯 % follow back xxbos # farmersprotest # xxmaj twitter xxmaj indian govt want xxmaj twitter to remove tweets / accounts which substantially supports xxmaj protest . i really needed to use aggressive words but aware of twitter violence … shame on your xxmaj
3,a root from them . xxmaj morcha is incomplete without the participation of our xxmaj women xxmaj farmers . xxmaj standing shoulder to shoulder in fight against tyranny . # farmersprotest # farmersprotest_standsstrong xxbos @ xxunk @ rahulgandhi xxmaj xxunk admi ko rafale ki sahi kimat nahi pata . xxmaj jo admi desh ki sena pe sawal uthata ho a … / / t . co / xxunk xxbos xxmaj people of,root from them . xxmaj morcha is incomplete without the participation of our xxmaj women xxmaj farmers . xxmaj standing shoulder to shoulder in fight against tyranny . # farmersprotest # farmersprotest_standsstrong xxbos @ xxunk @ rahulgandhi xxmaj xxunk admi ko rafale ki sahi kimat nahi pata . xxmaj jo admi desh ki sena pe sawal uthata ho a … / / t . co / xxunk xxbos xxmaj people of india
4,modidontsellfarmers xxbos xxmaj this done so divert the attention from real issues . # farmersprotest # farmersfightfascistmodi # farmlaws_threattoindians … xxbos you also forgot about your roots that you are from punjab . xxmaj shame on you . xxmaj having more than m followers you could have speak up in the support of farmers in xxmaj india but you did n’t ! # rihannasupportsindianfarmers # farmersprotest xxbos xxmaj our xxmaj farmer

In [31]:
# dls.vocab

In [32]:
learn = language_model_learner(dls, AWD_LSTM, metrics=accuracy,drop_mult=0.3)

In [33]:
learn.load('fine_tuned')

In [34]:
learn.save_encoder('finetuned')

In [215]:
dls_clas = TextDataLoaders.from_df(df=df, path=path,text_vocab=dls.vocab)

<h2> To calculate weights for F1 score</h2>
wj=n_samples / (n_classes * n_samplesj)

In [216]:
df['stance'].value_counts()

F    4279
N    1354
A     332
Name: stance, dtype: int64

In [217]:
df_test['stance'].value_counts()

F    1000
N    1000
A     332
Name: stance, dtype: int64

In [176]:
F_w = 2332/(3 * 1000)
N_w = 2332/(3 * 1000)
A_w = 2332/(3 * 332)
weights_tensor = torch.Tensor(np.array([A_w, F_w, N_w]))

In [218]:
F_w = 5965/(3 * 4279)
N_w = 5965/(3 * 1354)
A_w = 5965/(3 * 332)
weights_tensor = torch.Tensor(np.array([A_w, F_w, N_w]))

In [259]:
from sklearn.utils import class_weight


In [262]:
class_weights = class_weight.compute_class_weight('balanced',classes=np.unique(df['stance']),y=df['stance'])

In [264]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, loss_func=CrossEntropyLossFlat(weight=torch.Tensor(np.array(class_weights))),
                                drop_mult=0.5, 
                                metrics=[accuracy, F1Score(average='weighted')])
# learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy, F1Score(average='weighted')])

In [265]:
learn = learn.load_encoder('finetuned')

In [266]:
learn.fit_one_cycle(1, 2e-2,cbs=[EarlyStoppingCallback(monitor='f1_score', min_delta=0.01, patience=3)])

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,1.128125,1.081096,0.413244,0.484764,00:08


In [267]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2),cbs=[EarlyStoppingCallback(monitor='f1_score', min_delta=0.01, patience=3)])

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,1.089820,1.070740,0.553227,0.586018,00:09


In [268]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3),cbs=[EarlyStoppingCallback(monitor='f1_score', min_delta=0.01, patience=3)])

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,1.074760,1.103842,0.450126,0.508974,00:13


In [269]:
import gc
# del variables
gc.collect()
torch.cuda.empty_cache()

In [270]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(1e-3/(2.6**4),1e-3),cbs=[EarlyStoppingCallback(monitor='f1_score', min_delta=0.01, patience=3)])

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,1.018587,1.085986,0.483655,0.535607,00:19
1,1.006403,1.076599,0.523051,0.566123,00:19
2,0.986502,1.075906,0.472758,0.520259,00:19
3,0.934457,1.091653,0.504610,0.553359,00:19
4,0.870521,1.121152,0.507963,0.558572,00:19


No improvement since epoch 1: early stopping


In [271]:
learn.predict('A #Hindu MLA was stripped naked and beaten brutally in #Congress ruled #Punjab by so called farmers of #FarmersProtest.  Imagine the same, if had happened to a Sikh Or Muslim MLA or individual in a #BJP ruled state.  I know, you cant even imagine that...  Desh jal gaya hota')

('F', TensorText(1), TensorText([0.1183, 0.5028, 0.3789]))

In [272]:
learn.predict('Breaking Mobile tower = \'Modi\' s Public Development\'. 😂🤣Jio Bad boy, truth is slipping out of your tongue! 😝  Roads blocked & red fort indent, by the govt.   Trains- Rights of protest.   Opposition support-Democracy (only bit left)   #MonthsOfDelhiProtest  #FarmersProtest')

('F', TensorText(1), TensorText([0.3733, 0.3756, 0.2511]))

In [273]:
learn.predict('It\'s very hard to commute from Delhi to Ghaziabad daily due to roads blocked by farmers..   Till when common ppl will bear this ?? #Farmersprotest #Borderkhalikaro')

('F', TensorText(1), TensorText([0.4028, 0.4256, 0.1716]))

In [274]:
learn.predict('An elderly farmer beaten by police in Rohtak. This protest has seen too much blood & tears of innocent protestors and yet they remain resilient  #FarmersProtest  🚜🌾 ')

('F', TensorText(1), TensorText([0.1513, 0.6704, 0.1783]))

In [275]:
learn.predict('Shame on that bloody government that it is not listening the voice of poor farmers of this country  #FarmersProtest')

('A', TensorText(0), TensorText([0.5176, 0.3318, 0.1506]))

In [276]:
learn.predict('It\'s easy to feel helpless-like you can\'t fight the tide. But remember: small actions can have a huge impact,one person like u can inspire others to action #FarmersProtest   ::')

('F', TensorText(1), TensorText([0.3036, 0.5083, 0.1881]))

In [277]:
learn.predict('It\'s third time I\'ve felt such pain in this #FarmersProtest but it\'s part of the revolution. We\'ve to be strong and have to support each other like family. Maybe, youth and intellectual people should come forward. Please, don\'t waste your time, think what we can? #Lashkar_e_RSS ')

('F', TensorText(1), TensorText([0.2632, 0.5374, 0.1995]))

In [283]:
learn.predict('support the farmers.')

('N', TensorText(2), TensorText([0.2801, 0.2842, 0.4357]))

In [286]:
learn.predict('I stand with farmers!!!!')

('F', TensorText(1), TensorText([0.2877, 0.4196, 0.2927]))

In [280]:
learn.predict('Few antisocial & anti national elements already #FarmersProtestHijacked  They demand #Khalistan show involvement of #ISI & #PFI as well  We don\'t let them to make another #ShaheenBagh   Block #Delhi #BharatBandhOnDecember This is not #FarmersProtest  #SayNoToAnarchy ')

('A', TensorText(0), TensorText([0.7380, 0.1453, 0.1168]))

In [281]:
learn.predict('#FarmersProtest is going global as  extends support to farmers.  Don\'t be surprised if  writes an article against  …')

('F', TensorText(1), TensorText([0.1860, 0.6669, 0.1470]))

In [282]:
learn.predict('Bank unions lend support to farmers\' agitation against new farm laws  #FarmersProtest  #FarmLaws   YSXrNQ')

('F', TensorText(1), TensorText([0.2211, 0.7116, 0.0673]))